In [1]:
import json
import pandas as pd
import spacy
import numpy as np
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn import preprocessing
import random

In [2]:
d = pd.read_csv('reviews_gpt.csv', sep='\t')

In [6]:
d

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites,reviews.dateAdded,is_full,lang,reviews.text.tokens,reviews.length
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com,NaN,True,en,"['Our', 'experience', 'at', 'Rancho', 'Valenci...",37
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com,NaN,True,en,"['Amazing', 'place', '.', 'Everyone', 'was', '...",60
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com,NaN,True,en,"['We', 'booked', 'a', '3', 'night', 'stay', 'a...",93
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...,NaN,True,en,"['Currently', 'in', 'bed', 'writing', 'this', ...",41
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,Laurel,MD,kevan777,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...,NaN,True,en,"['I', 'stayed', 'here', 'with', 'my', 'family'...",46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14356,AWCPRUBCRxPSIh2RpqUY,2017-12-25T18:42:13Z,2018-12-03T09:04:10Z,2555 Hilton Garden Dr,"Hotels,Lodging,Hotel",Accommodation & Food Services,Auburn,US,us/al/auburn/2555hiltongardendr/-655190616,32.605260,...,Milton,UnitedStates,josephlL1691HG,http://www.yellowpages.com/auburn-al/mip/hilto...,http://hiltongardeninn.hilton.com/en/gi/hotels...,NaN,True,en,"['Staff', 'was', 'very', 'friendly', 'during',...",49
14357,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,...,Homer,AK,bobg187,https://www.tripadvisor.com/Hotel_Review-g5780...,https://ad.doubleclick.net/ddm/clk/317271033;1...,NaN,True,en,"['from', 'check', 'in', 'to', 'departure', ','...",53
14358,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,...,Conway,AR,K261ANbrendah,https://www.tripadvisor.com/Hotel_Review-g5780...,https://ad.doubleclick.net/ddm/clk/317271033;1...,NaN,True,en,"['This', 'Hampton', 'is', 'located', 'on', 'a'...",57
14359,AV1thTgM3-Khe5l_OvT5,2017-07-23T03:37:13Z,2018-12-02T00:27:00Z,7886 Main Street,"Hotels,Bar,Hotel,Restaurants",Accommodation & Food Services,Hunter,US,us/ny/hunter/7886mainstreet/-435384771,42.210915,...,Hunter,NewYork,soccerrocks2016,https://www.bbb.org/us/ny/hunter/profile/hotel...,http://www.roseberrysinnhunter.com/,NaN,True,en,"['Awesome', 'wings', '(', 'my', 'favorite', '

In [2]:
with open('hotel_sentiment_sentences.json', 'r', encoding='utf-8') as f:
    sentences = json.loads(f.read())

In [3]:
sentences = pd.DataFrame.from_dict(sentences, orient='index')
sentences = sentences.reset_index()
sentences = pd.melt(sentences, id_vars=['index'], value_vars=['pos', 'neg'], var_name='sentiment', value_name='sentence')
sentences = sentences.explode('sentence')
sentences = sentences.reset_index(drop=True)

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
def function(x):
    if pd.isna(x) == False:
        x = list(nlp(x))
        x = [token.text.lower() for token in x]
        return x
    else:
        return np.nan

In [6]:
sentences['tokens'] = sentences['sentence'].transform(lambda x: function(x))

KeyboardInterrupt: 

In [108]:
black_list = set(['i', 'we', 'he', 'she', 'our', 'his', 'her', 'him', 
                  'me', 'mine', 'my', 'myself', 'ourselves', 'herself', 'himself', 'you', 'your',
                 'sincerely', 'regards', 'answer', 'experience'])

In [109]:
def filtering(x):
    if isinstance(x, list):
        x = set(x)
        if black_list.isdisjoint(x):
            return True
        else:
            return False
    else:
        return np.nan

In [110]:
sentences['has_no_prons'] = sentences['tokens'].apply(lambda x: filtering(x))

In [111]:
sentences = sentences[sentences['has_no_prons'] == True].reset_index(drop=True)

In [112]:
statistics = sentences[['index', 'sentence']].groupby('index').count().sort_values(
    'sentence', ascending=False)

In [113]:
statistics[statistics['sentence'] > 14]

,sentence
index,
AVwcgshl_7pvs4fzv3ge,630
AWEKq1Eba4HuVbedMYk2,457
AVwdmGkmIN2L1WUfxXj-,435
AWM8kLxMRxPSIh2Ruhz4,432
AWE7bZc53-Khe5l_f3pJ,432
...,...
AVwdb_kt_7pvs4fz55eY,15
AVwdTTE3ByjofQCxnrAh,15
AWKjinxY3-Khe5l_huzI,15


In [114]:
sentences = sentences.sort_values('index').reset_index(drop=True)

In [115]:
sentences.to_csv('sentences_for_vectorization.csv', index=False, sep='\t')

#### Векторизация

In [116]:
vectorizer = TfidfVectorizer()
corpus = sentences['sentence'].tolist()
X = vectorizer.fit_transform(corpus)

In [117]:
corpus[0]

'the cheapest)Very easy to access the city, I5 and airport from here.'

In [118]:
X.shape

(37726, 12360)

In [119]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

In [120]:
vectors = []
for num, row in tqdm(sentences.iterrows()):
    tokens = row['tokens']
    vec = np.zeros(300,)
    for token in tokens:
        if (token in vectorizer.get_feature_names())and(token in model.vocab):
            column = vectorizer.get_feature_names().index(token)
            weight = X[num, column]
            vec += model[token] * weight
    vectors.append(vec)

0it [00:00, ?it/s]C:\Users\VADIK\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
37726it [49:03, 12.82it/s]


In [121]:
vectors = np.array(vectors)

In [123]:
#np.save('sentence_vectors', vectors)

#### K-means algorithm with elbow method

https://www.scikit-yb.org/en/latest/api/cluster/elbow.html

https://stats.stackexchange.com/questions/299013/cosine-distance-as-similarity-measure-in-kmeans

AVwcgshl_7pvs4fzv3ge

In [7]:
sentences = pd.read_csv('sentences_for_vectorization.csv', sep='\t')
vectors = np.load('sentence_vectors.npy')

In [17]:
reviews = dict()
for index in tqdm(sentences['index'].unique().tolist()):
    sample_sentences = sentences[sentences['index'] == index]
    if sample_sentences.shape[0] <= 10:
        review = ' '.join(sample_sentences['sentence'].tolist())
        reviews[index] = review
        continue
    indexes = sentences[sentences['index'] == index].index.tolist()
    sample = vectors[min(indexes): max(indexes) + 1]
    sample_norm = preprocessing.normalize(sample)
    model = KMeans()
    visualizer = KElbowVisualizer(model, k=(9, min([sample_sentences.shape[0], 30])))
    visualizer.fit(sample_norm)
    elbow_value = visualizer.elbow_value_
    if elbow_value == None:
        elbow_value = min(sample_sentences.shape[0], 30)
    kmeans = KMeans(n_clusters=elbow_value, init='random', random_state=42).fit(sample_norm)
    sample_sentences['class'] = kmeans.labels_
    review = []
    for num in range(0, elbow_value):
        review_sentences = sample_sentences[sample_sentences['class'] == num]['sentence'].tolist()
        if len(review_sentences) != 0:
            number = random.randint(0, len(review_sentences) - 1) 
            review.append(review_sentences[number])
    review = ' '.join(review)
    reviews[index] = review

  0%|                                                                                         | 0/2073 [00:00<?, ?it/s]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
  0%|                                                                               | 1/2073 [00:02<1:19:35,  2.30s/it]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:361: YellowbrickWarning: No 'knee' or 'elbow' point detected, p

C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:330: ConvergenceWarning: Number of distinct clusters (27) found smaller than n_clusters (29). Possibly due to duplicate points in X.
  self.estimator.fit(X, **kwargs)
<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
  2%|█▎                                                                            | 35/2073 [00:26<1:03:03,  1.86s/it]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\

  9%|███████▍                                                                       | 195/2073 [01:06<07:18,  4.28it/s]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 10%|███████▌                                                                       | 200/2073 [01:08<09:00,  3.46it/s]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:361: YellowbrickWarning: No 'knee' or 'elbow' point detected, p

 12%|█████████▎                                                                     | 243/2073 [01:48<44:12,  1.45s/it]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:361: YellowbrickWarning: No 'knee' or 'elbow' point detected, pass `locate_elbow=False` to remove the warning
  warnings.warn(warning_message, YellowbrickWarning)
<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 12%|█████████▎    

 13%|█████████▉                                                                     | 261/2073 [02:08<35:40,  1.18s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 13%|██████████                                                                     | 265/2073 [02:10<29:01,  1.04it/s]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 13%|██████████▏        

...

<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 96%|███████████████████████████████████████████████████████████████████████████▏  | 1999/2073 [10:09<02:01,  1.64s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 96%|███████████████████████████████████████████████████████████████████████████▎  | 2000/2073 [10:11<01:58,  1.62s/it]C:\Users\VADIK\anaconda3

 97%|███████████████████████████████████████████████████████████████████████████▉  | 2017/2073 [10:31<01:04,  1.14s/it]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:361: YellowbrickWarning: No 'knee' or 'elbow' point detected, pass `locate_elbow=False` to remove the warning
  warnings.warn(warning_message, YellowbrickWarning)
<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 97%|██████████████

 98%|████████████████████████████████████████████████████████████████████████████▍ | 2030/2073 [10:49<01:01,  1.44s/it]C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\utils\kneed.py:155: YellowbrickWarning: No 'knee' or 'elbow point' detected This could be due to bad clustering, no actual clusters being formed etc.
  warnings.warn(warning_message, YellowbrickWarning)
C:\Users\VADIK\anaconda3\lib\site-packages\yellowbrick\cluster\elbow.py:361: YellowbrickWarning: No 'knee' or 'elbow' point detected, pass `locate_elbow=False` to remove the warning
  warnings.warn(warning_message, YellowbrickWarning)
<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 98%|██████████████

 99%|█████████████████████████████████████████████████████████████████████████████ | 2047/2073 [11:14<00:46,  1.79s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 99%|█████████████████████████████████████████████████████████████████████████████ | 2048/2073 [11:17<00:51,  2.05s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
 99%|███████████████████

100%|█████████████████████████████████████████████████████████████████████████████▊| 2068/2073 [11:46<00:10,  2.14s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
100%|█████████████████████████████████████████████████████████████████████████████▉| 2070/2073 [11:48<00:05,  1.81s/it]<ipython-input-17-8fa0076bc903>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sentences['class'] = kmeans.labels_
100%|███████████████████

In [18]:
len(reviews)

2073

In [19]:
#with open('baseline_reviews.json', 'w', encoding='utf-8') as f:
#    f.write(json.dumps(reviews))